In [1]:
from rdkit import Chem
import rdkit.Chem.Lipinski as Lipinski
import rdkit.Chem.Crippen as Crippen
import pandas as pd
import numpy as np
import MDAnalysis as mda

In [5]:
mol = Chem.rdmolfiles.MolFromMol2File("../../dataset/refined-set/1a1e/1a1e_ligand.mol2")

In [6]:
#Hydrogen bond donors and acceptors in ligand https://www.rdkit.org/docs/source/rdkit.Chem.Lipinski.html
L_A = Lipinski.NumHAcceptors(mol)
L_D = Lipinski.NumHDonors(mol)

In [ ]:
print(L_A)
print(L_D)

In [8]:
#Octanol-water logP https://www.rdkit.org/docs/source/rdkit.Chem.Crippen.html
LogP = Crippen.MolLogP(mol) 

In [ ]:
print(LogP)

In [10]:
#Molar Refractivity https://www.rdkit.org/docs/source/rdkit.Chem.Crippen.html
MR = Crippen.MolMR(mol)

In [ ]:
print(MR)

In [12]:
#Wiener index http://www.scfbio-iitd.res.in/software/drugdesign/WINDEX/wienerindex.htm
def wiener_index(m):
    res = 0
    amat = Chem.GetDistanceMatrix(m)
    for i in range(m.GetNumAtoms()):
        for j in range(i+1,m.GetNumAtoms()):
            res += amat[i][j]
    return res

In [ ]:
#Wiener index for the ligand
wmol = wiener_index(mol)
print(wmol)


In [ ]:
#Molecular weight of the ligand https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html
MW = Chem.rdMolDescriptors.CalcExactMolWt(mol)
MW

In [116]:
#Calculate for all the ligands
indices = pd.read_csv("../../dataset/index_clean", delimiter=',',header=None, comment='#')

In [ ]:
indices[0].values

In [18]:
def wiener_index(m):
    res = 0
    amat = Chem.GetDistanceMatrix(m)
    for i in range(m.GetNumAtoms()):
        for j in range(i+1,m.GetNumAtoms()):
            res += amat[i][j]
    return res

In [ ]:
arr = np.zeros((len(indices[0]), 6))
for j,i in enumerate(indices[0]):
    #load sdf file
    #ignore errors
    try:
        mol = Chem.rdmolfiles.SDMolSupplier(f"../../dataset/refined-set/{i}/{i}_ligand.sdf")[0]
        MWe = Chem.rdMolDescriptors.CalcExactMolWt(mol)
        L_A = Lipinski.NumHAcceptors(mol)
        L_D = Lipinski.NumHDonors(mol)
        LogP = Crippen.MolLogP(mol) 
        MR = Crippen.MolMR(mol)
        wmol = wiener_index(mol)

    except:
        mol = Chem.rdmolfiles.MolFromMol2File(f"../../dataset/refined-set/{i}/{i}_ligand.mol2")  
        print("mol2")      
        MWe = Chem.rdMolDescriptors.CalcExactMolWt(mol)
        L_A = Lipinski.NumHAcceptors(mol)
        L_D = Lipinski.NumHDonors(mol)
        LogP = Crippen.MolLogP(mol) 
        MR = Crippen.MolMR(mol)
        wmol = wiener_index(mol)
    arr[j] = [MWe, L_A, L_D, LogP, MR, wmol]
  
    

In [ ]:
arr

In [ ]:
indices.shape

In [22]:
df = pd.DataFrame(arr, columns=['Molecular weight', 'Acceptors', 'Donors', 'LogP', 'Molecular Refractivity', 'Wiener index'])

In [23]:
df['Ids'] = indices[0]

In [ ]:
df['Molecular weight'].value_counts()

In [ ]:
df

In [2]:
import MDAnalysis as mda

In [28]:
protein = mda.Universe(f"../../dataset/refined-set/1a1e/1a1e_protein.pdb")

In [29]:
ligand = mda.Universe(f"../../dataset/refined-set/1a1e/1a1e_ligand.mol2")

In [ ]:
ligand.atoms[0]

In [31]:
complex = mda.Merge(protein.atoms, ligand.atoms)

In [32]:
ligname = ligand.residues[0].resname

In [33]:
pocket = complex.select_atoms(f"protein and around 6 resname {ligname}")

In [34]:
#Hydrogen bond donors an acceptors in protein https://www.imgt.org/IMGTeducation/Aide-memoire/_UK/aminoacids/charge/#:~:text=3%20amino%20acids%20(arginine%2C%20lysine,atoms%20in%20their%20side%20chain

In [35]:
#Select the hydrogen bond donors of arginine:

ARG_hbond = pocket.select_atoms("name NH1 NH2 NE")

In [36]:
#Select the hydrongen bond donors of the aminoacid groups as in the paper:

PD_amide = len(pocket.select_atoms("name N"))


In [37]:
#donors
PD_positive = len(pocket.select_atoms("name NZ NE NH1 NH2 ND1 NE2"))        #Lysine, arginine and histidine
PD_neutral = len(pocket.select_atoms("name ND2 NE2"))     #asparagine,glutamine
PD_heteroatom = len(pocket.select_atoms("name NE1"))    #Tryptophan
PD_OH = len(pocket.select_atoms("name OG OG1 OH"))    #Serine, Threonine, Tyrosine

In [38]:
#acceptors
PA_amide = len(pocket.select_atoms("name O"))
PA_negative = len(pocket.select_atoms("name OD1 OD2 OE1 OE2"))   #Aspartate, Glutamate
PA_neutral = len(pocket.select_atoms("name OE1 OG2 OH OD1"))       #Asparagine, Glutamine, tyrosine, serine
PA_aromatic = len(pocket.select_atoms("name OH ND1 NE2"))   #Tyrosine, Histidine

In [ ]:
#Construct a table with the logP and MR of every amino acid
amino_acid_names = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'] # resname 3letter

#List of amino acid SMILES
SMILES_ASP = "C([C@@H](C(=O)O)N)C(=O)O" #aspartate #https://pubchem.ncbi.nlm.nih.gov/compound/Aspartic-Acid
SMILES_ALA = "C[C@@H](C(=O)O)N" #Alanine
SMILES_ARG = "C(C[C@@H](C(=O)O)N)CN=C(N)N" #Arginine
SMILES_ASN = "C([C@@H](C(=O)O)N)C(=O)N" #Asparagine
SMILES_CYS = "C([C@@H](C(=O)O)N)S" #Cysteine
SMILES_GLN = "C(CC(=O)N)[C@@H](C(=O)O)N" #Glutamine
SMILES_GLU = "C(CC(=O)O)[C@@H](C(=O)O)N" #Glutamate
SMILES_GLY = "C(C(=O)O)N" #Glycine
SMILES_HIS = "C1=C(NC=N1)C[C@@H](C(=O)O)N" #Histidine
SMILES_ILE = "CC[C@H](C)[C@@H](C(=O)O)N" #Isoleucine
SMILES_LEU = "CC(C)C[C@@H](C(=O)O)N" #Leucine
SMILES_LYS = "C(CCN)C[C@@H](C(=O)O)N" #Lysine
SMILES_MET = "CSCC[C@@H](C(=O)O)N" #Methionine
SMILES_PHE = "C1=CC=C(C=C1)C[C@@H](C(=O)O)N" #Phenylalanine
SMILES_PRO = "C1C[C@H](NC1)C(=O)O" #Proline
SMILES_SER = "C([C@@H](C(=O)O)N)O" #Serine
SMILES_THR = "C[C@H]([C@@H](C(=O)O)N)O" #Threonine
SMILES_TRP = "C1=CC=C2C(=C1)C(=CN2)C[C@@H](C(=O)O)N" #Tryptophan
SMILES_TYR = "C1=CC(=CC=C1C[C@@H](C(=O)O)N)O" #Tyrosine
SMILES_VAL = "CC(C)[C@@H](C(=O)O)N" #Valine


#Mol object from SMILES
aspartic_mol = Chem.MolFromSmiles(SMILES_ASP) 
alanine_mol = Chem.MolFromSmiles(SMILES_ALA)
arginine_mol = Chem.MolFromSmiles(SMILES_ARG)
asparagine_mol = Chem.MolFromSmiles(SMILES_ASN)
cysteine_mol = Chem.MolFromSmiles(SMILES_CYS)
glutamine_mol = Chem.MolFromSmiles(SMILES_GLN)
glutamate_mol = Chem.MolFromSmiles(SMILES_GLU)
glycine_mol = Chem.MolFromSmiles(SMILES_GLY)
histidine_mol = Chem.MolFromSmiles(SMILES_HIS)
isoleucine_mol = Chem.MolFromSmiles(SMILES_ILE)
leucine_mol = Chem.MolFromSmiles(SMILES_LEU)
lysine_mol = Chem.MolFromSmiles(SMILES_LYS)
methionine_mol = Chem.MolFromSmiles(SMILES_MET)
phenylalanine_mol = Chem.MolFromSmiles(SMILES_PHE)
proline_mol = Chem.MolFromSmiles(SMILES_PRO)
serine_mol = Chem.MolFromSmiles(SMILES_SER)
threonine_mol = Chem.MolFromSmiles(SMILES_THR)
tryptophan_mol = Chem.MolFromSmiles(SMILES_TRP)
tyrosine_mol = Chem.MolFromSmiles(SMILES_TYR)
valine_mol = Chem.MolFromSmiles(SMILES_VAL)


In [ ]:
aspartic_mol # You can use this object to calculate the logP and MR of the amino acid

In [70]:

#Look up the smiles for every amino acid and calculate the logP and MR  and make a dictionary

#LogP
log_P_dict = {}
MR_dict = {}
logP_asp = Crippen.MolLogP(aspartic_mol)
logP_ala = Crippen.MolLogP(alanine_mol)
logP_arg = Crippen.MolLogP(arginine_mol)
logP_asn = Crippen.MolLogP(asparagine_mol)
logP_cys = Crippen.MolLogP(cysteine_mol)
logP_gln = Crippen.MolLogP(glutamine_mol)
logP_glu = Crippen.MolLogP(glutamate_mol)
logP_gly = Crippen.MolLogP(glycine_mol)
logP_his = Crippen.MolLogP(histidine_mol)
logP_ile = Crippen.MolLogP(isoleucine_mol)
logP_leu = Crippen.MolLogP(leucine_mol)
logP_lys = Crippen.MolLogP(lysine_mol)
logP_met = Crippen.MolLogP(methionine_mol)
logP_phe = Crippen.MolLogP(phenylalanine_mol)
logP_pro = Crippen.MolLogP(proline_mol)
logP_ser = Crippen.MolLogP(serine_mol)
logP_thr = Crippen.MolLogP(threonine_mol)
logP_trp = Crippen.MolLogP(tryptophan_mol)
logP_tyr = Crippen.MolLogP(tyrosine_mol)
logP_val = Crippen.MolLogP(valine_mol)

#MR
MR_asp = Crippen.MolMR(aspartic_mol)
MR_ala = Crippen.MolMR(alanine_mol)
MR_arg = Crippen.MolMR(arginine_mol)
MR_asn = Crippen.MolMR(asparagine_mol)
MR_cys = Crippen.MolMR(cysteine_mol)
MR_gln = Crippen.MolMR(glutamine_mol)
MR_glu = Crippen.MolMR(glutamate_mol)
MR_gly = Crippen.MolMR(glycine_mol)
MR_his = Crippen.MolMR(histidine_mol)
MR_ile = Crippen.MolMR(isoleucine_mol)
MR_leu = Crippen.MolMR(leucine_mol)
MR_lys = Crippen.MolMR(lysine_mol)
MR_met = Crippen.MolMR(methionine_mol)
MR_phe = Crippen.MolMR(phenylalanine_mol)
MR_pro = Crippen.MolMR(proline_mol)
MR_ser = Crippen.MolMR(serine_mol)
MR_thr = Crippen.MolMR(threonine_mol)
MR_trp = Crippen.MolMR(tryptophan_mol)
MR_tyr = Crippen.MolMR(tyrosine_mol)
MR_val = Crippen.MolMR(valine_mol)


log_P_dict['ASP'] = logP_asp
log_P_dict['ALA'] = logP_ala
log_P_dict['ARG'] = logP_arg
log_P_dict['ASN'] = logP_asn
log_P_dict['CYS'] = logP_cys
log_P_dict['GLN'] = logP_gln
log_P_dict['GLU'] = logP_glu
log_P_dict['GLY'] = logP_gly
log_P_dict['HIS'] = logP_his
log_P_dict['ILE'] = logP_ile
log_P_dict['LEU'] = logP_leu
log_P_dict['LYS'] = logP_lys
log_P_dict['MET'] = logP_met
log_P_dict['PHE'] = logP_phe
log_P_dict['PRO'] = logP_pro
log_P_dict['SER'] = logP_ser
log_P_dict['THR'] = logP_thr
log_P_dict['TRP'] = logP_trp
log_P_dict['TYR'] = logP_tyr
log_P_dict['VAL'] = logP_val

MR_dict['ASP'] = MR_asp
MR_dict['ALA'] = MR_ala
MR_dict['ARG'] = MR_arg
MR_dict['ASN'] = MR_asn
MR_dict['CYS'] = MR_cys
MR_dict['GLN'] = MR_gln
MR_dict['GLU'] = MR_glu
MR_dict['GLY'] = MR_gly
MR_dict['HIS'] = MR_his
MR_dict['ILE'] = MR_ile
MR_dict['LEU'] = MR_leu
MR_dict['LYS'] = MR_lys
MR_dict['MET'] = MR_met
MR_dict['PHE'] = MR_phe
MR_dict['PRO'] = MR_pro
MR_dict['SER'] = MR_ser
MR_dict['THR'] = MR_thr
MR_dict['TRP'] = MR_trp
MR_dict['TYR'] = MR_tyr
MR_dict['VAL'] = MR_val


In [ ]:
#Make a dictionary of the amino acids with their smiles
smiles_dict = {
    "ASP": "C([C@@H](C(=O)O)N)C(=O)O",
    "GLU": "C(CC(=O)O)[C@@H](C(=O)O)N",
    "LYS": "C(CCN)C[C@@H](C(=O)O)N",
    "ARG": "C(C[C@@H](C(=O)O)N)CN=C(N)N",
    "HIS": "C1=C(NC=N1)C[C@@H](C(=O)O)N",
    "SER": "C([C@@H](C(=O)O)N)O",
    "THR": "C[C@H]([C@@H](C(=O)O)N)O",
    "ASN": "C([C@@H](C(=O)O)N)C(=O)N",
    "GLN": "C(CC(=O)N)[C@@H](C(=O)O)N",
    "CYS": "C([C@@H](C(=O)O)N)S",
    "MET": "CSCC[C@@H](C(=O)O)N",
    "TYR": "C1=CC(=CC=C1C[C@@H](C(=O)O)N)O",
    "TRP": "C1=CC=C2C(=C1)C(=CN2)C[C@@H](C(=O)O)N",
    "PHE": "C1=CC=C(C=C1)C[C@@H](C(=O)O)N",
    "GLY": "C(C(=O)O)N",
    "PRO": "C1C[C@H](NC1)C(=O)O",
    "ALA": "C[C@@H](C(=O)O)N",
    "VAL": "CC(C)[C@@H](C(=O)O)N",
    "LEU": "CC(C)C[C@@H](C(=O)O)N",
    "ILE": "CC[C@H](C)[C@@H](C(=O)O)N"
}

In [ ]:
log_P_dict

In [ ]:
MR_dict

In [73]:
amino_acid_names_list = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']

In [ ]:
arr = np.zeros((len(indices[0]), 11))
for j,i in enumerate(indices[0]):
    protein = mda.Universe(f"../../dataset/refined-set/{i}/{i}_protein.pdb")
    ligand = mda.Universe(f"../../dataset/refined-set/{i}/{i}_ligand.mol2")

    ligname = 'LIG'
    for lgnm in ligand.residues:
        lgnm.resname = 'LIG'
    complex = mda.Merge(protein.atoms, ligand.atoms)
    pocket = complex.select_atoms(f"protein and around 6 resname {ligname}")
    PD_amide = len(pocket.select_atoms("name N"))
    PD_positive = len(pocket.select_atoms("name NZ NE NH1 NH2 ND1 NE2"))        #Lysine, arginine and histidine
    PD_neutral = len(pocket.select_atoms("name ND2 NE2"))     #asparagine,glutamine
    PD_heteroatom = len(pocket.select_atoms("name NE1"))    #Tryptophan
    PD_OH = len(pocket.select_atoms("name OG OG1 OH"))    #Serine, Threonine, Tyrosine
    PA_amide = len(pocket.select_atoms("name O"))
    PA_negative = len(pocket.select_atoms("name OD1 OD2 OE1 OE2"))   #Aspartate, Glutamate
    PA_neutral = len(pocket.select_atoms("name OE1 OG2 OH OD1"))       #Asparagine, Glutamine, tyrosine, serine
    PA_aromatic = len(pocket.select_atoms("name OH ND1 NE2"))   #Tyrosine, Histidine
    amino_acids = [residue.resname for residue in pocket.residues]
    P_logp = np.mean([log_P_dict[amino_acid] for amino_acid in amino_acids])
    P_MR = np.mean([MR_dict[amino_acid] for amino_acid in amino_acids])
    arr[j] = [PD_amide, PD_positive, PD_neutral, PD_heteroatom, PD_OH, PA_amide, PA_negative, PA_neutral, PA_aromatic, P_logp, P_MR]
    print(arr[j])
    


In [165]:
df['PD_amide'] = arr[:,0]
df['PD_positive'] = arr[:,1]
df['PD_neutral'] = arr[:,2]
df['PD_heteroatom'] = arr[:,3]
df['PD_OH'] = arr[:,4]
df['PA_amide'] = arr[:,5]
df['PA_negative'] = arr[:,6]
df['PA_neutral'] = arr[:,7]
df['PA_aromatic'] = arr[:,8]
df['P_logp'] = arr[:,9]
df['P_MR'] = arr[:,10]


In [ ]:
df

In [139]:
#Separate the number from str in indices[2] and assign to number and unit columns respectively
indices['number'] = [var[:-2] for var in indices[2]]
indices['unit'] = [var[-2:] for var in indices[2]]

In [ ]:
indices

In [ ]:
indices['unit'].value_counts()

In [145]:
#Create empty molar column
indices['Molar'] = np.nan


In [ ]:
#Convert the number to molar according to the unit
indices['number'] = indices['number'].astype(float)
indices['unit'] = indices['unit'].astype(str)
for i in range(len(indices)):
    if indices['unit'][i] == 'mM':
        indices['Molar'][i] = indices['number'][i]/1000
    elif indices['unit'][i] == 'nM':
        indices['Molar'][i] = indices['number'][i]/1000000000
    elif indices['unit'][i] == 'uM':
        indices['Molar'][i] = indices['number'][i]/1000000
    elif indices['unit'][i] == 'pM':
        indices['Molar'][i] = indices['number'][i]/1000000000000
    elif indices['unit'][i] == 'fM':
        indices['Molar'][i] = indices['number'][i]/1000000000000000
    else:
        indices['number'][i] = indices['number'][i]

In [148]:
#Convert the Molar to free energy in kcal/mol
indices['Molar'] = indices['Molar'].astype(float)
indices['Free energy'] = 0.592 * np.log(indices['Molar']) #RT = 0.592 kcal/mol at 298K

In [ ]:
indices

In [150]:
df['Free energy'] = indices['Free energy']

In [ ]:
df

In [168]:
df.to_csv("features.csv", index=False)

In [3]:
#Put your imports here so you don't have to re run the whole script
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.svm import SVR
import seaborn as sns
import sklearn
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# sudo pip install xgboost
import xgboost

warnings.filterwarnings('ignore')

In [ ]:
df

In [46]:
#Try implementing SVM (https://medium.com/@niousha.rf/support-vector-regressor-theory-and-coding-exercise-in-python-ca6a7dfda927), LR (https://www.geeksforgeeks.org/python-linear-regression-using-sklearn/), RF (https://www.geeksforgeeks.org/random-forest-regression-in-python/), and XGBoost (https://machinelearningmastery.com/xgboost-for-regression/)
#You can load the calculated features with df = pd.read_csv("features.csv") and then use the features to train the models
#The target will be the Free energy column, the rest will be the features
#reduces overfitting

df = pd.read_csv("features.csv")

X = df.drop(['Free energy', 'Ids'], axis=1)
Y = df['Free energy']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

svm_preds = {}

#Scales X-data, fit() method expects a 2D array-like input
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

svr_lin = SVR(kernel = 'linear')
svr_rbf = SVR(kernel = 'rbf')
svr_poly = SVR(kernel = 'poly')

svr_lin.fit(X_train_scaled, y_train)
svr_rbf.fit(X_train_scaled, y_train)
svr_poly.fit(X_train_scaled, y_train)

svm_preds['linear_svr_pred'] = svr_lin.predict(X_test_scaled)
svm_preds['rbf_svr_pred'] = svr_rbf.predict(X_test_scaled)
svm_preds['poly_svr_pred'] = svr_poly.predict(X_test_scaled)




In [ ]:
from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score

print("Linear SVR")
print('Mean Squared Error:', mean_squared_error(y_test, svm_preds['linear_svr_pred']))
print('R2 Score:', r2_score(y_test, svm_preds['linear_svr_pred']))
print("\n")
print("RBF SVR")
print('Mean Squared Error:', mean_squared_error(y_test, svm_preds['rbf_svr_pred']))
print('R2 Score:', r2_score(y_test, svm_preds['rbf_svr_pred']))
print("\n")
print("Poly SVR")
print('Mean Squared Error:', mean_squared_error(y_test, svm_preds['poly_svr_pred']))
print('R2 Score:', r2_score(y_test, svm_preds['poly_svr_pred']))


In [ ]:
plt.plot(y_test, svm_preds['linear_svr_pred'], 'o')
plt.xlabel('True Free energy')
plt.ylabel('Predicted Free energy')

In [ ]:
plt.plot(y_test, svm_preds['rbf_svr_pred'], 'o')
plt.xlabel('True Free energy')
plt.ylabel('Predicted Free energy')

In [ ]:
plt.plot(y_test, svm_preds['poly_svr_pred'], 'o')
plt.xlabel('True Free energy')
plt.ylabel('Predicted Free energy')

In [ ]:
#Try implementing LR (https://www.geeksforgeeks.org/python-linear-regression-using-sklearn/), 
#You can load the calculated features with df = pd.read_csv("features.csv") and then use the features to train the models
#The target will be the Free energy column, the rest will be the features

df = pd.read_csv("features.csv")

X = df.drop(['Free energy', 'Ids'], axis=1)
Y = df['Free energy']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1, 1)
X_test = np.array(X_test)
y_test = np.array(y_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


logit = LinearRegression()
logit.fit(X_train, y_train)
y_test_pred = logit.predict(X_test)

np.mean(y_test_pred == y_test)

In [ ]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('R2 Score:', metrics.r2_score(y_test, y_test_pred))

In [ ]:
plt.plot(y_test, y_test_pred, 'o')
plt.xlabel('True Free energy')
plt.ylabel('Predicted Free energy')

In [42]:
#Try implementing, RF (https://www.geeksforgeeks.org/random-forest-regression-in-python/) 
#You can load the calculated features with df = pd.read_csv("features.csv") and then use the features to train the models
#The target will be the Free energy column, the rest will be the features
#combines multiple "trees"

df = pd.read_csv("features.csv")

X = df.drop(['Free energy', 'Ids'], axis=1)
Y = df['Free energy']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
label_encoder = LabelEncoder()
x_categorical = df.select_dtypes(include=['object']).apply(label_encoder.fit_transform)
x_numerical = df.select_dtypes(exclude=['object']).values
x = pd.concat([pd.DataFrame(x_numerical), x_categorical], axis=1).values

# Fitting Random Forest Regression to the dataset
regressor = RandomForestRegressor(n_estimators=10, random_state=0, oob_score=True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Fit the regressor with x and y data
regressor.fit(X_train, y_train)

y_test_pred = regressor.predict(X_test)



In [ ]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('R2 Score:', metrics.r2_score(y_test, y_test_pred))

In [ ]:
plt.plot(y_test, y_test_pred, 'o')
plt.xlabel('True Free energy')
plt.ylabel('Predicted Free energy')

In [56]:
#Try implementing XGBoost (https://machinelearningmastery.com/xgboost-for-regression/)
#You can load the calculated features with df = pd.read_csv("features.csv") and then use the features to train the models
#The target will be the Free energy column, the rest will be the features


df = pd.read_csv("features.csv")
X_test, X_train, y_test, y_train = train_test_split(df.drop(['Free energy', 'Ids'], axis=1), df['Free energy'], test_size=0.3, random_state=0)
scaler = StandardScaler()   
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = XGBRegressor()
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)






In [ ]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('R2 Score:', metrics.r2_score(y_test, y_test_pred))

In [ ]:

plt.plot(y_test, y_test_pred, 'o')
plt.xlabel('True Free energy')
plt.ylabel('Predicted Free energy')
plt.plot([np.min(y_test), np.max(y_test)], [np.min(y_test_pred), np.max(y_test_pred)], 'k--')

In [4]:
from keras.callbacks import ModelCheckpoint, History
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf
tf.config.list_physical_devices('GPU')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2024-11-03 16:51:24.570204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-03 16:51:24.658756: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-03 16:51:24.683144: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 16:51:24.869132: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-03 16:51:26.527405: W tensorflow/compiler/tf2

In [9]:
df = pd.read_csv("features.csv")
X_test, X_train, y_test, y_train = train_test_split(df.drop(['Free energy', 'Ids'], axis=1), df['Free energy'], test_size=0.3, random_state=0)
scaler = StandardScaler()   
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [41]:
NN_model = Sequential()
#NN_model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
NN_model.add(Dense(600, activation='relu'))
#NN_model.add(Dense(250, activation='relu'))
NN_model.add(Dense(1, activation='linear'))
NN_model.compile(optimizer = "adam", loss = 'mse')

# tf.keras.layers.Dense(300, activation=tf.nn.relu),
# tf.keras.layers.Dense(250, activation=tf.nn.relu),
# tf.keras.layers.Dense(235, activation=tf.nn.relu),
# tf.keras.layers.Dense(200, activation=tf.nn.relu),
# tf.keras.layers.Dense(125, activation=tf.nn.relu),
# tf.keras.layers.Dense(75, activation=tf.nn.relu),



In [ ]:
history = NN_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split = 0.2)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

In [ ]:
print('Best validation loss:', np.min(history.history['val_loss']), np.argmin(history.history['val_loss']))

In [ ]:
y_test_pred = NN_model.predict(X_test)

In [ ]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('R2 Score:', metrics.r2_score(y_test, y_test_pred))

In [ ]:
#keras tuner check this tutorial https://haneulkim.medium.com/hyperparameter-tuning-with-keras-tuner-full-tutorial-f8128397e857

In [5]:
import kerastuner as kt

In [6]:
def build_model(hp):
    num_layers = hp.Int('num_layers', min_value=1, max_value=3, step=1)
    units_per_layer = hp.Int('units_per_layer', min_value=32, max_value=512, step=32)
    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.4,step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model = Sequential()
    for i in range(num_layers):
        model.add(Dense(units=units_per_layer, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [ ]:
obj = kt.Objective('val_loss', direction='min')
tuner = kt.RandomSearch(build_model, objective=obj, max_trials=500, executions_per_trial=1,
                               project_name="Hyper tuning", directory="tuning")

In [ ]:
tuner.search(X_train, y_train, epochs=60, validation_split=0.2)

In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(best_hps.values)

In [11]:
best_model = tuner.hypermodel.build(best_hps)

In [32]:
NN_model = Sequential()
NN_model.add(Dense(192, input_dim=X_train.shape[1], activation='relu'))
NN_model.add(Dropout(0.2))
NN_model.add(Dense(192, activation='relu'))
NN_model.add(Dropout(0.2))
NN_model.add(Dense(192, activation='relu'))
NN_model.add(Dropout(0.2))
NN_model.add(Dense(1, activation='linear'))
NN_model.compile(optimizer =Adam(learning_rate=0.0001), loss = 'mse')

In [33]:
callbacks = [ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', save_best_only=True)]

In [ ]:
history = NN_model.fit(X_train, y_train, epochs=500, validation_split=0.2, callbacks=callbacks)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
print('Best validation loss:', np.min(history.history['val_loss']), np.argmin(history.history['val_loss']))
plt.legend()
plt.show()

In [36]:
NN_model.load_weights('best_model.keras')

In [ ]:
y_pred = NN_model.predict(X_test)

In [ ]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('R2 Score:', metrics.r2_score(y_test, y_pred))

In [ ]:
plt.plot(y_test, y_pred, 'o')
plt.xlabel('True Free energy')
plt.ylabel('Predicted Free energy')
plt.show()

In [10]:
df

,Molecular weight,Acceptors,Donors,LogP,Molecular Refractivity,Wiener index,Ids,PD_amide,PD_positive,PD_neutral,PD_heteroatom,PD_OH,PA_amide,PA_negative,PA_neutral,PA_aromatic,Free energy,P_logp,P_MR
0,231.170319,2.0,3.0,-0.13160,60.5356,461.0,2tpi,24.0,0.0,0.0,0.0,5.0,23.0,5.0,4.0,1.0,-5.874825,-0.665612,29.440791
1,147.112804,2.0,3.0,-0.85160,36.0603,130.0,4tln,6.0,9.0,4.0,0.0,2.0,6.0,6.0,4.0,7.0,-5.072544,-0.403714,34.962933
2,363.057999,10.0,6.0,-2.56970,76.4808,1230.0,1rnt,9.0,6.0,6.0,0.0,3.0,9.0,9.0,10.0,7.0,-7.070675,-0.619947,36.111405
3,182.081170,2.0,3.0,-0.37020,46.1227,268.0,4ts1,11.0,4.0,5.0,1.0,4.0,12.0,11.0,10.0,5.0,-6.727787,-0.498970,31.695033
4,519.213437,5.0,5.0,3.26060,135.1412,4249.0,4tmn,16.0,9.0,5.0,1.0,6.0,16.0,13.0,10.0,9.0,-13.859616,-0.503965,34.127585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5311,253.050394,3.0,2.0,1.21620,64.9371,478.0,6d1i,6.0,9.0,5.0,0.0,0.0,5.0,4.0,2.0,8.0,-5.577268,-0.570889,33.634874
5312,212.025563,3.0,1.0,0.53470,50.5625,260.0,6uh0,5.0,7.0,6.0,1.0,3.0,6.0,5.0,5.0,8.0,-8.272838,-0.393956,34.876296
5313,404.151622,6.0,1.0,4.72902,114.4347,2134.0,6k04,10.0,2.0,2.0,1.0,2.0,9.0,3.0,4.0,4.0,-10.960875,-0.186890,36.583310
5314,436.991713,8.0,5.0,0.42410,89.9438,1701.0,6ic2,8.0,7.0,6.0,1.0,3.0,10.0,4.0,4.0,8.0,-8.229820,-0.330392,33.531504


In [11]:
#Data agumentation
df_aug = df.copy()


In [ ]:
zeroarray = np.zeros((5316, 1))
#5316
print(zeroarray
      )

In [16]:
for col in df_aug.columns:
    if col[0] == "P": 
        df_aug[col] = zeroarray
        
    


In [18]:
df_aug['Free energy'] = zeroarray

In [20]:
df_aug.to_csv("features_aug.csv", index=False)

In [21]:
#merge dataframes
df = pd.read_csv("features.csv")
df_aug = pd.read_csv("features_aug.csv")
df_merged = pd.concat([df, df_aug], axis=0)

In [23]:
df_merged.reset_index(drop=True, inplace=True)

In [24]:
df_merged

,Molecular weight,Acceptors,Donors,LogP,Molecular Refractivity,Wiener index,Ids,PD_amide,PD_positive,PD_neutral,PD_heteroatom,PD_OH,PA_amide,PA_negative,PA_neutral,PA_aromatic,Free energy,P_logp,P_MR
0,231.170319,2.0,3.0,-0.13160,60.5356,461.0,2tpi,24.0,0.0,0.0,0.0,5.0,23.0,5.0,4.0,1.0,-5.874825,-0.665612,29.440791
1,147.112804,2.0,3.0,-0.85160,36.0603,130.0,4tln,6.0,9.0,4.0,0.0,2.0,6.0,6.0,4.0,7.0,-5.072544,-0.403714,34.962933
2,363.057999,10.0,6.0,-2.56970,76.4808,1230.0,1rnt,9.0,6.0,6.0,0.0,3.0,9.0,9.0,10.0,7.0,-7.070675,-0.619947,36.111405
3,182.081170,2.0,3.0,-0.37020,46.1227,268.0,4ts1,11.0,4.0,5.0,1.0,4.0,12.0,11.0,10.0,5.0,-6.727787,-0.498970,31.695033
4,519.213437,5.0,5.0,3.26060,135.1412,4249.0,4tmn,16.0,9.0,5.0,1.0,6.0,16.0,13.0,10.0,9.0,-13.859616,-0.503965,34.127585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10627,253.050394,3.0,2.0,1.21620,64.9371,478.0,6d1i,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
10628,212.025563,3.0,1.0,0.53470,50.5625,260.0,6uh0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
10629,404.151622,6.0,1.0,4.72902,114.4347,2134.0,6k04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
10630,436.991713,8.0,5.0,0.42410,89.9438,1701.0,6ic2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [ ]:
#Repeat the ML training with the augmented data (df_merged)